<a href="https://colab.research.google.com/github/sangwook-hub/DataAnalysis/blob/master/Neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import load_img, img_to_array, save_img 
from keras import backend as K
import numpy as np
from keras.applications import vgg19
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.optimize import fmin_l_bfgs_b
import time

In [ ]:
content_path = "/MyDrive/My Drive/Neural/content_image/pic_3.jpeg"
style_path = "/MyDrive/My Drive/Neural/style_image/calliefink_crop.jpg"

width, height = load_img(content_path).size

img_height = 400
img_width = int(width * img_height / height)

In [ ]:
def preprocess_image(image_path):
  img = load_img(image_path)
  img = img_to_array(img)
  # 차원을 늘려준다.
  # (428, 640, 3) ==> (1, 428, 640, 3)
  img = np.expand_dims(img,axis=0) 
  draw_img = img.copy()

  # 이미지 인코딩
  img = vgg19.preprocess_input(img)
  return img, draw_img

#imagenet의 평균값을 더 해주기
def deprocess_image(x):
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 3] += 123.68
  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [ ]:
content_img = cv2.imread(content_path)
print("content_img shape:",content_img.shape)

img, draw_img = preprocess_image(content_path)
print("draw_img shape:",draw_img.shape)

print("img shape:",img.shape)

In [ ]:
img = img.reshape(img.shape[1], img.shape[2], 3)

plt.figure(figsize = (10,10))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height, img_width))
  img = img_to_array(img)
  # 차원을 늘려준다.
  # (428, 640, 3) ==> (1, 428, 640, 3)
  img = np.expand_dims(img,axis=0) 

  # 이미지 인코딩
  img = vgg19.preprocess_input(img)
  return img

# ImageNet의 평균 픽셀 값을 더 하는 함수
def deprocess_image(x):
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  
  #BGR == > RGB
  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [ ]:
img = preprocess_image(content_path)
img = deprocess_image(img)
print(img.shape)
img = img.reshape(img.shape[1],img.shape[2],img.shape[3])

plt.figure(figsize = (10,10))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
## Tensor 배열 만들기
content_image = K.constant(preprocess_image(content_path))
style_image = K.constant(preprocess_image(style_path))


#다른 Tensor 할당
combination_image = K.placeholder((1,img_height, img_width,3))

input_tensor = K.concatenate([content_image,
                              style_image,
                              combination_image], axis=0)

model = vgg19.VGG19(input_tensor= input_tensor,
                    weights = 'imagenet',
                    include_top = False)

print("모델 학습 완료")

In [ ]:
def content_loss(base, combination):
  return K.sum(K.square(combination - base))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features,K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

def total_variation_loss(x):
  a = K.square(
      x[:,:img_height -1, :img_width -1, :] - x[:, 1:, :img_width -1, :])
  b = K.square(
      x[:,:img_height -1, :img_width -1, :] - x[:,:img_height -1, 1:, :])
  return K.sum(K.pow(a + b, 1.25))

In [ ]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
outputs_dict

In [ ]:
## 콘텐츠 손실에 사용할 층(상위 층)
content_layer = 'block5_conv2'

##스타일 손실에 사용할 층
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block4_conv1']

# 손실 항목의 가중치 평균에 사용할 가중치
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.) #모든 손실함수의 값을 하나의 손실 값으로 정의할 손실을 지정

layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0,:,:,:]
combination_features = layer_features[2:,:,:]

loss =  loss + content_weight * content_loss(target_image_features, 
                                      combination_features)

for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1,:,:,:]
  combination_features = layer_features[2,:,:,:]
  sl = style_loss(style_reference_features, combination_features)
  loss = loss + ((style_weight / len(style_layers)) * sl)

loss = loss + (total_variation_weight * total_variation_loss(combination_image))

In [ ]:
# 손실에 대한 생성된 이미지의 그래디언트를 구합니다
grads = K.gradients(loss, combination_image)[0]

# 현재 손실과 그래디언트의 값을 추출하는 케라스 Function 객체입니다
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [ ]:
result_prefix = 'style_transfer_result'
iterations = 1500

x = preprocess_image(content_path)
x = x.flatten()
loss_list = []
for i in range(iterations):
  print('반복 횟수:',i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                   fprime = evaluator.grads, maxfun = 20)
  print('현재 손실 값:',min_val)
  loss_list.append(min_val)

  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_image(img)
  fname = "/content/drive/My Drive/Neural_Style_transfer/output_images/" + result_prefix + '_at_iteration_%d.png' % (i)
  end_time = time.time()
  print('%d 번째 반복 완료: %ds' % (i, end_time - start_time))
  print("")

save_img(fname, img)